# Intro

本文区分光栅化渲染和体渲染的viewport transformation过程。

光栅化渲染将3D三角面投影到2D坐标下，为了模拟现实中近大远小的效果，需要先做透视投影，再计算每个射线的颜色；体渲染直接在3D空间采样射线上的多个位置来计算射线颜色，因此不需要透视投影。

两者都约定相机坐标下的相机在原点，并且相机坐标轴由$\mathbf{v}_f,\mathbf{v}_u,\mathbf{v}_r$三个轴定义，其中$\mathbf{v}_f$与相机面向的方向相反，$\mathbf{v}_u$为相机向上的方向，$\mathbf{v}_r=\mathbf{v}_f\times \mathbf{v}_u$（右手定则）。



# 光栅化渲染相机变换
### 投影

<img src="../pasteImage/perspective_projection_1.png">



首先定义视锥体，渲染内容只包含视锥体内部的内容（即近平面和远平面中间的内容），如上图所示（绿色的为近平面）。

一般视锥体有关的定义有：
- 近平面左下角$(l,b,n)$和右上角$(r,t,n)$两点，一般有约定$l=-r,b=-t$。
- 长宽比$\mathrm{aspect}=r/t$
- 定义垂直方向视野角度fovy

根据以上定义（结合上图）显然有：
$$
\begin{aligned}
\tan{\frac{\mathrm{fovy}}{2}}&=\frac{t}{-n}\\
\mathrm{aspect}&=\frac{r}{t}
\end{aligned}
$$
所以给定$\mathrm{fovy},\mathrm{aspect},n$就相当于给定$l,r,b,t$。

投影变换首先希望将视锥体和射线扭曲成平行线，方便随后的正交投影，如下图（a,b）所示。透视投影前的$\mathbf{P}=(x,y,z)$在扭曲后变成了$\mathbf{P'}=(x',y',z')$，这个变化可以用一个矩阵$P'=\mathbf{M}_{\mathrm{persp}\rightarrow\mathrm{orth}}$表示：
$$
\mathbf{M}_{\mathrm{persp}\rightarrow\mathrm{orth}}=
\begin{pmatrix}
n &  &  & \\
& n & & \\
 &  & n+f & -nf\\
 &  & 1 & \\
\end{pmatrix}
$$
暂不介绍$\mathbf{M}_{\mathrm{persp}\rightarrow \mathrm{orth}}$的推导过程，因为我讲不明白。



<img src="../pasteImage/perspective_projection_2.png">



正交投影先把视锥体中心平移到原点，再放缩进canonical cube里，最后取消z轴即可。因为整个过程只涉及到平移和放缩，所以$\mathbf{M}_\mathrm{orth}$如下：
$$
\mathbf{M}_\mathrm{orth}=
\begin{pmatrix}
\frac{2}{r-l} &  &  & \\
& \frac{2}{t-b} & & \\
 &  & \frac{2}{n-f} & \\
 &  &  & 1\\
\end{pmatrix}
\begin{pmatrix}
1 &  &  & -\frac{r+l}{2}\\
& 1 & & -\frac{t+b}{2}\\
 &  & 1 & -\frac{n+f}{2}\\
 &  & 1 & \\
\end{pmatrix}
$$

整个投影过程为$\mathbf{M}_{\mathrm{persp}\rightarrow\mathrm{orth}}\mathbf{M}_\mathrm{orth}$：
$$
\mathbf{M}_\mathrm{persp}=
\begin{pmatrix}
\frac{2}{r-l} &  &  & \\
& \frac{2}{t-b} & & \\
 &  & \frac{2}{n-f} & \\
 &  &  & 1\\
\end{pmatrix}
\begin{pmatrix}
1 &  &  & -\frac{r+l}{2}\\
& 1 & & -\frac{t+b}{2}\\
 &  & 1 & -\frac{n+f}{2}\\
 &  &  & 1\\
\end{pmatrix}
\begin{pmatrix}
n &  &  & \\
& n & & \\
 &  & n+f & -nf\\
 &  & 1 & \\
\end{pmatrix}
=
\begin{pmatrix}
\frac{n}{r} &  &  & \\
& \frac{n}{t} & & \\
 &  & \frac{n+f}{n-f} & \frac{-2nf}{n-f}\\
 &  & 1 & \\
\end{pmatrix}
=
\begin{pmatrix}
\frac{1}{\tan{\frac{\mathrm{fovy}}{2}}*\mathrm{aspect}} &  &  & \\
& \frac{1}{\tan{\frac{\mathrm{fovy}}{2}}} & & \\
 &  & \frac{n+f}{n-f} & \frac{-2nf}{n-f}\\
 &  & 1 & \\
\end{pmatrix}
$$

注意到上式和OpenGL的glFrustum里的矩阵不一样，因为此矩阵里的$u,f$为距离值（正值）而非坐标值；而上式子里的$u,f$为带符号坐标值（负值）。我们不妨令$f'=-f,n'=-n$，并且乘一个翻转z轴的缩放矩阵。

$$
\mathbf{M}_\mathrm{persp}=
\begin{pmatrix}
1 &  &  & \\
& 1 & & \\
 &  & -1 &\\
 &  &  & 1\\
\end{pmatrix}
\begin{pmatrix}
-\frac{n'}{r} &  &  & \\
& -\frac{n'}{t} & & \\
 &  & -\frac{n'+f'}{f'-n'} & -\frac{2n'f'}{f'-n'}\\
 &  & 1 & \\
\end{pmatrix}
=
\begin{pmatrix}
-\frac{n'}{r} &  &  & \\
& -\frac{n'}{t} & & \\
 &  & \frac{n'+f'}{f'-n'} & \frac{2n'f'}{f'-n'}\\
 &  & 1 & \\
\end{pmatrix}
$$
不妨整体乘以-1，因为齐次坐标同时乘以一个系数和原来的坐标是一样的，于是我们得到和OpenGL相同的投影矩阵：
$$
\mathbf{M}_\mathrm{persp}=
\begin{pmatrix}
\frac{n'}{r} &  &  & \\
& \frac{n'}{t} & & \\
 &  & -\frac{n'+f'}{f'-n'} & -\frac{2n'f'}{f'-n'}\\
 &  & -1 & \\
\end{pmatrix}
$$

### 旋转
旋转过程和体渲染旋转过程一致，此处略过旋转矩阵推导。最终的投影结果为：$\mathbf{M}_\mathrm{persp}\mathbf{R}_{w2c}$。

# 体渲染相机变换
<img src="../pasteImage/world_camera_image.png">

### 旋转
**约定**：相机坐标下的相机位置永远在原点，并且相机坐标轴由$\mathbf{v}_f,\mathbf{v}_u,\mathbf{v}_r$三个轴定义，其中$\mathbf{v}_f$与相机面向的方向相反，$\mathbf{v}_u$为相机向上的方向，$\mathbf{v}_r=\mathbf{v}_f\times \mathbf{v}_u$（右手定则）。

假设相机在世界坐标$\mathbf{c}=(c_x,c_y,c_z)$下。

$$
\begin{equation}
\begin{split}
\mathbf{v}_{f}=\mathbf{v}_{forward}&=\frac{\mathbf{c}}{\lVert\mathbf{c}\rVert _2^2}\\
\mathbf{v}_{r}=\mathbf{v}_{right}&=\mathbf{v}_{f}\times\mathbf{y}\\
\mathbf{v}_{u}=\mathbf{v}_{up}&=\mathbf{v}_{r}\times \mathbf{v}_{forward}\\
\end{split}
\end{equation}
$$
假设给定世界坐标$\mathbf{x}_w$，想要求得其相机坐标$\mathbf{x}_c=(a,b,d)^T$。我们有：
$$
a\mathbf{v}_r+b\mathbf{v}_u+d\mathbf{v}_f+\mathbf{c}=\mathbf{x}_w
$$
即
$$
\begin{equation}
\begin{pmatrix}
| & | & | & |\\
\mathbf{v}_{r} & \mathbf{v}_{u} & \mathbf{v}_{f} & \mathbf{c}\\
| & | & | & |\\
0 & 0 & 0 & 1\\
\end{pmatrix}
\mathbf{x}_c=\mathbf{x}_w
\end{equation}
$$
即
$$
\begin{equation}
\mathbf{R}_{c2w}=
\begin{pmatrix}
| & | & | & |\\
\mathbf{v}_{r} & \mathbf{v}_{u} & \mathbf{v}_{f} & \mathbf{c}\\
| & | & | & |\\
0 & 0 & 0 & 1\\
\end{pmatrix}
\end{equation}
$$
那么
$$
\mathbf{R}_{w2c}=\mathbf{R}_{c2w}^{-1}
$$
其中llff数据集```transform.json```里的```frames-->transform_matrix```就是$\mathbf{R}_{c2w}$。

### 投影

此时$\mathbf{P}(x,y,z)$为camera coordinate下的坐标，$f$为focal length，因为$\mathbf{p}(u,v)$和$\mathbf{P}(x,y,z)$在一条线上，所以满足
$$
\begin{equation}
u=f\frac{x}{z}\quad v=f\frac{y}{z}
\end{equation}
$$


因为像素坐标是原点在边角，所以需要平移原点：
$$
\begin{equation}
u=f\frac{x}{z}+d_u\quad v=f\frac{y}{z}+d_v
\end{equation}
$$

假设真实的画布长宽$(H\times W)$，我们不妨用$Hu$和$Wv$代替$u\in\mathbb{R}$和$v\in\mathbb{R}$，此时$u,v\in\mathbb{N}$：
$$
\begin{equation}
Hu=f\frac{x}{z}+d_u\quad Wv=f\frac{y}{z}+d_v
\end{equation}
$$

等价于：
$$
\begin{equation}
u=\alpha_x\frac{x}{z}+d_x\quad y=\alpha_y\frac{y}{z}+d_y
\end{equation}
$$

整理成矩阵形式：
$$
\begin{equation}
z\mathbf{p}=
z\underbrace{\begin{pmatrix}
u\\
v\\
1
\end{pmatrix}}_{\mathbf{p}}
=\underbrace{\begin{pmatrix}
\alpha_x & 0 & d_x\\
0 & \alpha_y & d_y\\
0 & 0 & 1 \\
\end{pmatrix}}_{\mathbf{K}}
\underbrace{
\begin{pmatrix}
x\\
y\\
z
\end{pmatrix}}_{\mathbf{P}}
\end{equation}
$$

由于传感器没有垂直于光轴，传感器轴之间可能发生倾斜，$\mathbf{K}$中加入了$s$来代表这样的倾斜：
$$
\begin{equation}
\mathbf{K}=
\begin{pmatrix}
\alpha_x & s & d_x\\
0 & \alpha_y & d_y\\
0 & 0 & 1 \\
\end{pmatrix}
\end{equation}
$$

其中$\mathbf{K}$又常称为camera intrinsic matrix

<!-- ### Ray生成

<img src="../pasteImage/ray_generate.png"> -->